In [13]:
import functools

#  #!/bin/bash
#  today=$(date +'%Y-%m-%d')
#  now=$(date '+%Y-%m-%d %H:%M:%S.%6N')
#  pass_file="_route_codes.csv.bz2"
#  file=$today$pass_file
#  filename=$(echo "$file" |cut -d'.' -f 1)
#  kinit hdfs-hdp_deptrans@MOS.RU -kt /etc/security/keytabs/hdfs.headless.keytab
#  if [ ! -f /data/sftpshare/sftpclient4/files/dictionaries/$file ]
#  then
#    echo -e "$filename\t$now\t0\t0\tNULL\tNULL" >> /integrations/mgt/logs/mgt_general_log.txt
#  else
#    bzip2 -c -d /data/sftpshare/sftpclient4/files/dictionaries/$file | hdfs dfs -put - /user/root/tmp/mosgortrans/dictionaries/
#    hdfs dfs -mv /user/root/tmp/mosgortrans/dictionaries/- /user/root/tmp/mosgortrans/dictionaries/$filename
#    export SPARK_MAJOR_VERSION=2
#    echo "$filename starts"
#    spark-submit /integrations/mgt/routes_codes_integration.py $filename
#    hdfs dfs -rm -skipTrash /user/root/tmp/mosgortrans/dictionaries/$filename
#  fi

#  # =================================================

#  #! /usr/bin/env python
#  # -*- coding: utf-8 -*-

#  import sys
#  import os
#  import csv
#  import re
#  from pyspark.sql import SparkSession, HiveContext, SQLContext
#  from pyspark import SparkConf,SparkContext
#  from pyspark.sql.functions import unix_timestamp, from_unixtime, UserDefinedFunction
#  from pyspark.sql.types import StringType
#  from datetime import datetime

#  spark = SparkSession.builder.enableHiveSupport().getOrCreate()
#  hivecontext = HiveContext(spark.sparkContext)
#  hivecontext.setConf("hive.exec.dynamic.partition", "true")
#  hivecontext.setConf("hive.exec.dynamic.partition.mode", "nonstrict")
#  sqlcontext = SQLContext(spark.sparkContext)

#  def int_check(row, chislo, column):
#      if re.match('^-?[0-9]+$',chislo):
#          return "no_remove"
#      else:
#          error_description = 'Некорректный формат числа в поле {}'.format(column)
#          write_process_log(row, error_description)
#          return "remove"
        
#  def string_check(row, text, column):
#      if str(text.encode('utf-8')) in ('Null', 'null', 'NULL', 'None', ''):
#          error_description = 'Пропущено значение в поле {}'.format(column)
#          write_process_log(row, error_description)
#          return "remove"
#      else:
#          return "no_remove"

#  def row_check(x):
#      remove_flag = list()
#      new_list = list(x)
#      if len(x)==3:
#          remove_flag.append(int_check(x,new_list[0], 'id'))
#          remove_flag.append(string_check(x,new_list[1], 'short_name'))
#          remove_flag.append(string_check(x,new_list[2], 'route_code'))
#      else:
#          error_description = 'Количество атрибутов не совпадает со спецификацией. В строке {} атрибутов'.format(len(x))
#          write_process_log(x, error_description)
#          remove_flag.append("remove")
#      if "remove" in remove_flag:
#          new_list.append("remove")
#      else:
#          new_list.append("no_remove")
#      return new_list

#  def write_process_log(row, error, filename = sys.argv[1]):
#      writerow = ','.join(row)
#      if os.path.isfile('/integrations/mgt/logs/log_{}_route_codes.txt'.format(filename.split('_')[0]))==True:
#          with open('/integrations/mgt/logs/log_{}_route_codes.txt'.format(filename.split('_')[0]), 'a') as f:
#              writer = csv.writer(f, delimiter = '\t')
#              writer.writerow((unicode(filename, 'utf-8').encode('utf-8'),writerow.encode('utf-8'),unicode(error, 'utf-8').encode('utf-8')))
#      else:
#          headerlist = (u'Название файла', u'Строка', u'Описание ошибки')
#          with open('/integrations/mgt/logs/log_{}_route_codes.txt'.format(filename.split('_')[0]), 'a') as f:
#              writer = csv.writer(f, delimiter = '\t')
#              writer.writerow(([i.encode('utf-8') for i in headerlist]))
#              writer.writerow((unicode(filename, 'utf-8').encode('utf-8'),writerow.encode('utf-8'),unicode(error, 'utf-8').encode('utf-8')))

#  def write_general_log(log):
#      if os.path.isfile('/integrations/mgt/logs/mgt_general_log.txt')==True:
#          with open('/integrations/mgt/logs/mgt_general_log.txt','a') as f:
#              writer = csv.writer(f, delimiter = '\t')
#              writer.writerow((log['Название файла']
#                              ,log['Дата и время начала проверки']
#                              ,log['Всего строк']
#                              ,log['Корректных строк']
#                              ,log['Некорректных строк']
#                              ,log['Дата и время записи данных в Hive']
#                              ,log['Общее время выполнения проверки и загрузки данных']))
#      else:
#          with open('/integrations/mgt/logs/mgt_general_log.txt','a') as f:
#              writer = csv.writer(f, delimiter = '\t')
#              writer.writerow(('Название файла','Дата и время начала проверки','Всего строк','Корректных строк' \
#                               ,'Некорректных строк','Дата и время записи данных в Hive'\
#                               ,'Общее время выполнения проверки и загрузки данных'))
#              writer.writerow((log['Название файла']
#                              ,log['Дата и время начала проверки']
#                              ,log['Всего строк']
#                              ,log['Корректных строк']
#                              ,log['Некорректных строк']
#                              ,log['Дата и время записи данных в Hive']
#                              ,log['Общее время выполнения проверки и загрузки данных']))

#  def write_hive(final_rdd):
#      route_df = final_rdd.toDF(['id','short_name', 'full_name'])
#      route_df = route_df.selectExpr("CAST(id as bigint) as id",
#                                 "short_name",
#                                 "full_name")
#      route_df.write.insertInto("MOSGORTR_DATA.route_codes", overwrite=True)

#  def main():
#      start_time = datetime.now()
#      lines = spark.sparkContext.textFile("hdfs://hdp-master2.mos.ru:8020/user/root/tmp/mosgortrans/dictionaries/{}".format(sys.argv[1]))
#      lines = lines.map(lambda x: x.split(';'))
#      lines_count = lines.count()
#      checked_lines = lines.map(lambda x: row_check(x))
#      remove_number = checked_lines.filter(lambda x: x[-1]=='remove').count()
#      final_rdd = checked_lines.filter(lambda x: x[-1]=='no_remove').map(lambda x: x[:-1])
#      write_number = final_rdd.count()
#      write_hive(final_rdd)
#      end_time = datetime.now()
#      delta = end_time - start_time
#      log_record = {'Название файла': sys.argv[1]
#                   ,'Дата и время начала проверки' : start_time
#                   ,'Всего строк' : lines_count
#                   ,'Корректных строк' : write_number
#                   ,'Некорректных строк' : remove_number
#                   ,'Дата и время записи данных в Hive' : end_time
#                   ,'Общее время выполнения проверки и загрузки данных' : delta}
#      write_general_log(log_record)
#      spark.stop()

#  if __name__ == '__main__':
#      main()

print("ok")

ok


In [14]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
# !conda install -y pyspark
import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf "spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234 -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell"
os.environ['SPARK_HOME'] = "/usr/local/spark/spark-2.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/user/.sdkman/candidates/java/current"

print("ok")

ok


In [15]:
def main():
    WORKDIR = "/home/user/HR/RAIF/BOOSTERS"
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    try:
        spark = SparkSession.builder.master("local[*]").getOrCreate()
        sc = SQLContext(spark.sparkContext)
        import pandas as pd
        pandas_df = pd.read_csv(WORKDIR + '/test_set.csv')  # assuming the file contains a header
        print(pandas_df)
        # s_df = sc.createDataFrame(pandas_df)
        # print(s_df)
    finally:
        spark.stop()

if __name__ == '__main__':
    main()

print("ok")

/home/user/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


           amount                                     atm_address  \
0        2.211818                                             NaN   
1        1.331379                                             NaN   
2        2.608004                                             NaN   
3        1.916752                                             NaN   
4        1.981067                                             NaN   
5        2.389274                                             NaN   
6        2.004232                                             NaN   
7        2.101059                                             NaN   
8        1.634257                                             NaN   
9        1.902229                                             NaN   
10       4.014786                                             NaN   
11       2.615034                                             NaN   
12       2.711650                                             NaN   
13       1.938108                 

ok
